In [ ]:
using MatrixProductBP, MatrixProductBP.Models
using Plots
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using JLD2
using LaTeXStrings

include("../../telegram/notifications.jl");

In [ ]:
T = 200         # final time
k = 3          # degree
m⁰ = 0.3       # magnetization at time zero

β = 1.0
J = 0.6
h = 0.0;

In [ ]:
wᵢ = fill(HomogeneousGlauberFactor(J, h, β), T+1)
ϕᵢ = [ t == 0 ? [(1+m⁰)/2, (1-m⁰)/2] : ones(2) for t in 0:T]
bp = mpbp_infinite_graph(k, wᵢ, 2, ϕᵢ)
cb = CB_BP(bp);

In [ ]:
matrix_sizes = [10, 15, 20, 30, 40]
maxiters = fill(70, length(matrix_sizes))
maxiters = [70, 70, 70, 70, 50]
iters = zeros(Int, length(maxiters))
tol = 1e-5
for i in eachindex(maxiters)
    iters[i], _ = iterate!(bp; maxiter=maxiters[i], svd_trunc=TruncBond(matrix_sizes[i]), cb, tol)
end

In [ ]:
iters_cum = cumsum(iters)
inds = 1:iters_cum[1]
pl = plot(inds, cb.Δs[inds], label="$(matrix_sizes[1]) matrices")
for i in 2:length(iters)
    inds = iters_cum[i-1]:iters_cum[i]
   plot!(pl, inds, cb.Δs[inds], label="$(matrix_sizes[i]) matrices")
end
plot(pl, ylabel="convergence error", xlabel="iters", yaxis=:log10, size=(500,300),
    legend=:outertopright)

In [ ]:
spin(x, i) = 3-2x
spin(x) = spin(x, 0)
m = only(means(spin, bp));

In [ ]:
m_ss, = equilibrium_observables(RandomRegular(k), J; β, h)

In [ ]:
blue = theme_palette(:auto)[1]
pl = plot(0:T, map(spin, only(cb.m[end])), m=:o, xlabel="time", ylabel="magnetization", label="MPBP",
    size=(500,300), xticks=0:20:T, ms=3, title="Glauber infinite $k-regular", titlefontsize=12,
    legend=:bottomright, msc=:auto, c=blue)
hline!(pl, [m_ss, -m_ss], ls=:dash, label="equilib")

In [ ]:
@telegram "glauber infinite"

## Monte Carlo

In [ ]:
using Graphs, IndexedGraphs, Statistics

N = 10^3
g = random_regular_graph(N, k)
ising = Ising(IndexedGraph(g); J=fill(J, ne(g)), h=fill(h, N), β)
bp_mc = mpbp(Glauber(ising, T); ϕ = fill(ϕᵢ, N))
sms = SoftMarginSampler(bp_mc);

In [ ]:
sample!(sms, 10^3)
spin(x) = 3-2x
m_mc = mean(vec(spin.(mean(X, dims=1))) for X in sms.X);

In [ ]:
scatter!(deepcopy(pl), 0:T, m_mc, m=:diamond, c=:black, size=(500,300), label="MonteCarlo")

In [ ]:
using ColorSchemes
cg = cgrad(:matter, length(matrix_sizes)+1, categorical=true)
pl_iters = plot(; xlabel="t",ylabel="magnetiz", legend=:outerbottomright, size=(900,400),
    title="Glauber close to transition (J=$J, h=$h), RRG k=$k", margin=5Plots.mm)
for i in eachindex(matrix_sizes)
   plot!(pl_iters, 0:T, map(spin, only(cb.m[iters_cum[i]])), label="MPBP bond dim $(matrix_sizes[i])",
        c=cg[i+1]) 
end
hline!(pl_iters, [m_ss], label="equilib", ls=:dash)
scatter!(pl_iters, 0:T, m_mc, m=:o, ms=2, c=:black, label="MonteCarlo")

In [ ]:
jldsave("../data/glauber_infinite_graph.jld2"; T, J, h, k, matrix_sizes, cb, iters_cum, m_ss, m_mc);

## Larger T

In [ ]:
# T2 = 250         # final time
# k = 3          # degree
# m⁰ = 0.3       # magnetization at time zero

# β = 1.0
# J = 0.6
# h = 0.0;

In [ ]:
# wᵢ2 = fill(HomogeneousGlauberFactor(J, h, β), T2+1)
# ϕᵢ2 = [ t == 0 ? [(1+m⁰)/2, (1-m⁰)/2] : ones(2) for t in 0:T2]
# bp2 = mpbp_infinite_graph(k, wᵢ2, 2, ϕᵢ2)
# cb2 = CB_BP(bp2);

In [ ]:
# matrix_sizes = [10, 15, 20, 30, 40]
# maxiters = fill(70, length(matrix_sizes))
# maxiters = [70, 70, 70, 70, 50]
# iters2 = zeros(Int, length(maxiters))
# tol = 1e-5
# for i in eachindex(maxiters)
#     iters2[i], _ = iterate!(bp2; maxiter=maxiters[i], svd_trunc=TruncBond(matrix_sizes[i]), cb=cb2, tol)
# end

In [ ]:
# iters_cum2 = cumsum(iters2)
# inds2 = 1:iters_cum2[1]
# pl2 = plot(inds, cb.Δs[inds2], label="$(matrix_sizes[1]) matrices")
# for i in 2:length(iters2)
#     inds = iters_cum2[i-1]:iters_cum2[i]
#    plot!(pl2, inds, cb2.Δs[inds], label="$(matrix_sizes[i]) matrices")
# end
# plot(pl2, ylabel="convergence error", xlabel="iters", yaxis=:log10, size=(500,300),
#     legend=:outertopright)

In [ ]:
# using ColorSchemes
# cg = cgrad(:matter, length(matrix_sizes)+1, categorical=true)
# pl_iters2 = plot(; xlabel="t",ylabel="magnetiz", legend=:outerbottomright, size=(900,400),
#     title="Glauber close to transition (J=$J, h=$h), RRG k=$k", margin=5Plots.mm)
# for i in eachindex(matrix_sizes)
#    plot!(pl_iters2, 0:T2, map(spin, only(cb2.m[iters_cum2[i]])), label="MPBP bond dim $(matrix_sizes[i])",
#         c=cg[i+1]) 
# end
# hline!(pl_iters, [m_ss], label="equilib", ls=:dash)
# scatter!(pl_iters, 0:T, m_mc, m=:o, ms=2, c=:black, label="MonteCarlo")

In [ ]:
# @telegram "glauber infinite"